In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

# pyspark DataFrame

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

print(spark.version)

3.4.3


In [4]:
# 샘플 데이터 생성
data = [
    ("Alice", 34, "New York"),
    ("Bob", 45, "San Francisco"),
    ("Catherine", 29, "Los Angeles"),
    ("David", 40, "Chicago"),
    ("Eva", 37, "New York")
]

# 데이터 프레임 생성
columns = ["Name", "Age", "City"]
df = spark.createDataFrame(data, columns)

# 데이터 프레임 표시
df.show()


+---------+---+-------------+
|     Name|Age|         City|
+---------+---+-------------+
|    Alice| 34|     New York|
|      Bob| 45|San Francisco|
|Catherine| 29|  Los Angeles|
|    David| 40|      Chicago|
|      Eva| 37|     New York|
+---------+---+-------------+



In [5]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- City: string (nullable = true)



In [6]:
df_filtered = df.filter(df.Age >= 35)

In [7]:
df_filtered.show()

+-----+---+-------------+
| Name|Age|         City|
+-----+---+-------------+
|  Bob| 45|San Francisco|
|David| 40|      Chicago|
|  Eva| 37|     New York|
+-----+---+-------------+



In [8]:
df_grouped = df.groupBy("City").avg("Age")
df_grouped.show()

+-------------+--------+
|         City|avg(Age)|
+-------------+--------+
|San Francisco|    45.0|
|     New York|    35.5|
|  Los Angeles|    29.0|
|      Chicago|    40.0|
+-------------+--------+



# pyspark JSON

In [37]:
import json


# JSON 데이터 생성
data = [
    {"name": "Alice", "age": 34, "city": "New York"},
    {"name": "Bob", "age": 45, "city": "San Francisco"},
    {"name": "Catherine", "age": 29, "city": "Los Angeles"},
    {"name": "David", "age": 40, "city": "Chicago"},
    {"name": "Eva", "age": 37, "city": "New York"}
]
json_data = json.dumps(data, indent=4)

rdd = sc.textFile("/user/rsh/poverty.csv")

for line in rdd.take(10):
    print(line)


date,values,realtime_start,realtime_end,state,id,title,frequency_short,units_short,seasonal_adjustment_short
1989-01-01,10.6,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of All Ages in Poverty for Alaska,A,%,NSA
1990-01-01,,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of All Ages in Poverty for Alaska,A,%,NSA
1991-01-01,,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of All Ages in Poverty for Alaska,A,%,NSA
1992-01-01,,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of All Ages in Poverty for Alaska,A,%,NSA
1993-01-01,11.2,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of All Ages in Poverty for Alaska,A,%,NSA
1994-01-01,,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of All Ages in Poverty for Alaska,A,%,NSA
1995-01-01,10.1,2023-01-21,2023-01-21,Alaska,PPAAAK02000A156NCEN,Estimated Percent of People of A

In [23]:
dir(rdd.collect())

['__add__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

# Spark 실습

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [2]:
df = spark.\
    read.\
    csv("/encore/20240614/civilian_force.csv",
        header=True,)

In [3]:
df.schema

StructType([StructField('date', StringType(), True), StructField('values', StringType(), True), StructField('realtime_start', StringType(), True), StructField('realtime_end', StringType(), True), StructField('state', StringType(), True), StructField('id', StringType(), True), StructField('title', StringType(), True), StructField('frequency_short', StringType(), True), StructField('units_short', StringType(), True), StructField('seasonal_adjustment_short', StringType(), True)])

In [104]:
df2 = spark.\
    read.\
    csv("hdfs://192.168.0.160:50070/encore/20240614/civilian_force.csv",
        header=True,
        inferSchema=True)

24/06/17 06:50:11 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: hdfs://192.168.0.160:50070/encore/20240614/civilian_force.csv.
java.net.SocketException: Call From producer/192.168.56.123 to namenode:50070 failed on socket exception: java.net.SocketException: Network is unreachable; For more details see:  http://wiki.apache.org/hadoop/SocketException
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:871)
	at org.apache.hadoop.ipc.Client.getRp

Py4JJavaError: An error occurred while calling o621.csv.
: java.net.SocketException: Call From producer/192.168.56.123 to namenode:50070 failed on socket exception: java.net.SocketException: Network is unreachable; For more details see:  http://wiki.apache.org/hadoop/SocketException
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:871)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ipc.Client.call(Client.

In [85]:
set(df2.schema) - set(df.schema)

{StructField('date', DateType(), True),
 StructField('realtime_end', DateType(), True),
 StructField('realtime_start', DateType(), True),
 StructField('values', DoubleType(), True)}

In [94]:
df2.show()

24/06/17 05:47:55 ERROR Executor: Exception in task 0.0 in stage 49.0 (TID 60)
java.net.SocketException: Call From producer/192.168.56.123 to namenode:8020 failed on socket exception: java.net.SocketException: Network is unreachable; For more details see:  http://wiki.apache.org/hadoop/SocketException
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:871)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1616)
	at org.apache.hadoop.ipc.Client.call(Client.java:1558)
	at org.apache.hadoop.ipc.

Py4JJavaError: An error occurred while calling o518.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 49.0 failed 1 times, most recent failure: Lost task 0.0 in stage 49.0 (TID 60) (producer executor driver): java.net.SocketException: Call From producer/192.168.56.123 to namenode:8020 failed on socket exception: java.net.SocketException: Network is unreachable; For more details see:  http://wiki.apache.org/hadoop/SocketException
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:913)
	at org.apache.hadoop.net.NetUtils.wrap